In [1]:
!pip install -q llama-index llama-index-core llama-index-embeddings-huggingface llama-index-vector-stores-qdrant qdrant-client sentence-transformers

In [2]:
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    Settings
)
from llama_index.core.node_parser import SentenceWindowNodeParser
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.vector_stores.qdrant import QdrantVectorStore
from qdrant_client import QdrantClient

In [3]:
Settings.embed_model = HuggingFaceEmbedding(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

In [4]:
Settings.node_parser = SentenceWindowNodeParser.from_defaults(
    window_size=3,
    window_metadata_key="window",
    original_text_metadata_key="original_text",
)

In [5]:
# 🚫 No LLM set
Settings.llm = None

LLM is explicitly disabled. Using MockLLM.


In [6]:
qdrant_client = QdrantClient(
    path="./qdrant_data"
)

In [7]:
vector_store = QdrantVectorStore(
    client=qdrant_client,
    collection_name="sentence_window_index",
)

In [8]:
Settings.vector_store = vector_store

In [9]:
documents = SimpleDirectoryReader("/content/data").load_data()

In [10]:
index = VectorStoreIndex.from_documents(documents)

In [11]:
retriever = index.as_retriever(similarity_top_k=2)

In [12]:
query = "What specific economic and diplomatic actions did the United States take in response to Russia’s invasion of Ukraine?"

In [13]:
nodes = retriever.retrieve(query)

In [14]:
nodes

[NodeWithScore(node=TextNode(id_='37211874-5ba1-4cd7-a2d1-05c73cd903bd', embedding=None, metadata={'file_path': '/content/data/state_of_the_union.txt', 'file_name': 'state_of_the_union.txt', 'file_type': 'text/plain', 'file_size': 39027, 'creation_date': '2025-12-28', 'last_modified_date': '2025-12-28', 'window': 'It matters.  American diplomacy matters.  American resolve matters. \n\n Putin’s latest attack on Ukraine was premeditated and unprovoked. \n\n He rejected repeated efforts at diplomacy. \n\n He thought the West and NATO wouldn’t respond.  And he thought he could divide us at home. ', 'original_text': 'Putin’s latest attack on Ukraine was premeditated and unprovoked. \n\n'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date', 'window', 'original_text'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date', 'window', 'original_

In [15]:
def pretty_print_sentence_windows(nodes):
    for i, n in enumerate(nodes, 1):
        print("=" * 80)
        print(f"RESULT {i}")
        print(f"Score: {n.score:.3f}\n")

        print("🎯 Matched sentence:")
        print(n.node.text.strip())

        print("\n🪟 Context window:")
        print(n.node.metadata["window"].strip())

In [16]:
from llama_index.core.postprocessor import MetadataReplacementPostProcessor

query_engine = index.as_query_engine( similarity_top_k=2,
                                     verbose=True,
                                     node_postprocessors = [MetadataReplacementPostProcessor(target_metadata_key="window")]
                                  )


In [17]:
nodes2 = query_engine.query(query)

In [18]:
nodes2

Response(response='Context information is below.\n---------------------\nfile_path: /content/data/state_of_the_union.txt\n\nIt matters.  American diplomacy matters.  American resolve matters. \n\n Putin’s latest attack on Ukraine was premeditated and unprovoked. \n\n He rejected repeated efforts at diplomacy. \n\n He thought the West and NATO wouldn’t respond.  And he thought he could divide us at home.\n\nfile_path: /content/data/state_of_the_union.txt\n\nAnd a proud Ukrainian people, who have known 30 years  of independence, have repeatedly shown that they will not tolerate anyone who tries to take their country backwards.  \n\n To all Americans, I will be honest with you, as I’ve always promised.  A Russian dictator, invading a foreign country, has costs around the world. \n\n And I’m taking robust action to make sure the pain of our sanctions  is targeted at Russia’s economy.  And I will use every tool at our disposal to protect American businesses and consumers. \n\n Tonight, I ca

In [19]:
for i, n in enumerate(nodes2.source_nodes, 1):
    print("=" * 80)
    print(f"Result {i} | Score: {n.score:.3f}")
    print(n.node.text.strip())

Result 1 | Score: 0.474
It matters.  American diplomacy matters.  American resolve matters. 

 Putin’s latest attack on Ukraine was premeditated and unprovoked. 

 He rejected repeated efforts at diplomacy. 

 He thought the West and NATO wouldn’t respond.  And he thought he could divide us at home.
Result 2 | Score: 0.453
And a proud Ukrainian people, who have known 30 years  of independence, have repeatedly shown that they will not tolerate anyone who tries to take their country backwards.  

 To all Americans, I will be honest with you, as I’ve always promised.  A Russian dictator, invading a foreign country, has costs around the world. 

 And I’m taking robust action to make sure the pain of our sanctions  is targeted at Russia’s economy.  And I will use every tool at our disposal to protect American businesses and consumers. 

 Tonight, I can announce that the United States has worked with 30 other countries to release 60 Million barrels of oil from reserves around the world.  

 